# 라이브러리 로드

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm


# 데이터 로드

In [5]:
data_url = "https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt"

data = pd.read_table(data_url, names=['score', "reviews"])

# 데이터 확인

In [6]:
data.head()

,score,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [7]:
data.tail()

,score,reviews
199995,2,장마라그런가!!! 달지않아요
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,5,넘이쁘고 쎄련되보이네요~
199999,5,아직 사용해보지도않았고 다른 제품을 써본적이없어서 잘 모르겠지만 ㅎㅎ 배송은 빨랐습니다


In [8]:
data.shape

(200000, 2)

총 20만개의 데이터로 구성되어있는 것을 확인할 수 있습니다.

# 데이터 전처리

## 중복 데이터 확인

In [9]:
data[data.duplicated(['reviews'])]

,score,reviews
10958,5,2022년까지 넉넉함
20907,1,배송이느려 페인트칠 다하고 왔네요
49674,1,배송이 너무 늦어요
52247,2,생각보다 별로네요~
63566,2,아직 잘 모르겠어요...
...,...,...
195857,2,딱 돈만큼 하는 제품
196291,2,배송이 느려요
197220,1,정사이즈보다 조금더 크게시켜야할듯
198330,2,배송도 빠르고 좋습니다


92개의 데이터가 중복되어 있는 것을 확인할 수 있습니다.

## 중복데이터 제거

In [10]:
data = data.drop_duplicates(['reviews'])
data.shape

(199908, 2)

## 결측치 확인

In [11]:
data.isnull().sum()

score      0
reviews    0
dtype: int64

결측치는 없는것으로 확인할 수 있습니다.

# 학습/테스트 데이터 분리

텍스트 전처리하기 이전에 학습/테스트 데이터를 분리하겠습니다.

In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.25, random_state=42)
train.shape, test.shape

((149931, 2), (49977, 2))

In [13]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

# 텍스트 전처리

텍스트의 띄어쓰기와 맞춤법을 보정하기 위하여 py-hanspell,pykospacing 라이브러리를 설치 후에 사용하겠습니다.

## py-hanspell 간단한 예시 사용

In [18]:
from hanspell import spell_checker
original_text = '안녕하세요.저는한국인입니다.이문장은한글로작성됬습니다.'
result = spell_checker.check(original_text)
print("변경 전 : ", original_text)
print("변경 후 : ",result.checked)

변경 전 :  안녕하세요.저는한국인입니다.이문장은한글로작성됬습니다.
변경 후 :  안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.


## pykospacing 간단한 예시 사용


In [19]:
from pykospacing import Spacing
spacing = Spacing()
original_text = '안녕하세요.저는한국인입니다.이문장은한글로작성됬습니다.'
result = spacing(original_text)
print("변경 전 : ", original_text)
print("변경 후 : ",result)

변경 전 :  안녕하세요.저는한국인입니다.이문장은한글로작성됬습니다.
변경 후 :  안녕하세요. 저는 한국인입니다.이 문장은 한글로 작성 됬습니다.


## 띄어쓰기 및 맞춤법 보정

py-hanspell 라이브러리에서 띄어쓰기과 맞춤법을 보정해주기 때문에  
원본 reviews의 띄어쓰기를 모두 제거해주겠습니다.

또한 pykosapcing 라이브러리를 사용하여 py-hanspell 사용시 에러가 나는 텍스트에 대하여는 띄어쓰기만 진행해주겠습니다.

In [20]:
def remove_spaces(x):
  x = x.replace(" ","")
  return x

train['reviews'] = train['reviews'].apply(remove_spaces)
train['reviews']

0         사이즈를3센치씩늘린건데도작아요그리고색상은완전달라요칙칙한핑크네요ㅠㅠ많이아쉽지만암막효과...
1                                 ㅂ불만족..빗이아픔..멍이피부에빗질못해주겟네요
2         이제품쓰고삼일만에변기물이잘안내려갔어요.혹시나해서다시빼보니물이다시잘내려가네요..,이많...
3                                                 적당하고만족합니다
4                                  편하자고이용하는밀키튼데손은근많이가서저는패쓰요
                                ...                        
149926    그냥그래요ㄷㄷㄷㄷㅂㅂㅂㅂ그냥그래요ㄷㄷㄷㄷㅂㅂㅂㅂㅁㅁㅁㅁㅋㅋㅋㅋㅌㅌㅌㅌㅍㅍㅍㅍㄹㄹㄹㄹ...
149927       비싸요진짜별거아니게허접하게생겼는데이게이만원이라니진짜가방구겨지는것만아니면구매안했음!!
149928                                   100장주문안됩니다500장가능해요
149929            .하림치킨여기서구입하니엄청저렴하네요배송도쾅꽝얼어서도착하고아주만족합니다!ㅋㅋ
149930                                     조금약해보이는데저렴하게잘삿어요
Name: reviews, Length: 149931, dtype: object

훈련 데이터가 모두 띄어쓰기가 제거되어있는것을 확인할 수 있습니다.  
또한 테스트 데이터 또한 모두 제거해주겠습니다.

In [21]:
test['reviews'] = test['reviews'].apply(remove_spaces)
test['reviews']

0                                          너무낮고솜도적고실망스럽습니다
1                                            피부에뾰루지가많이올라와요
2                    배송도빠르네요가격대비좋은것같아요~~~첨에는힘들어하나조금지나니잘하네요
3                                     재구매입니다.핏도좋고착용감도좋습니다.
4                      파손제품온거출장같다오늘받았는데현재상황장난하시는지택배회사항의하세요
                               ...                        
49972    연결해서쓰려고3단으로2개주문했는데..연결부위맞추기가넘힘드네여거기다세로조립품은연결하는...
49973            허리는너무크고종아리는꽉끼어서약간불편해하네요아들이그래도따뜻하다며잘입고다닙니다
49974       샵에가서받았었는데이거구매하고집에서셀프로하는데넘나좋으네요^^*세트로사길잘한것같아요ㅎㅎ
49975          가격도착하고다좋은데조금얇은감이있네요넓이도조금커서살짝헐렁거리지만그래도만족합니다~
49976                                           아~정말이건아닌듯ㅎ
Name: reviews, Length: 49977, dtype: object

py-hanspell과 pykospacing을 사용하여 reviews를 보정해주겠습니다.  
먼저 py-hanspell 라이브러리를 사용하여 맞춤범 및 띄어쓰기 보정을 진행하고  
에러가 나는 부분은 pykospacing 사용하여 띄어쓰기만 진행하겠습니다.  
하지만 그럼에도 또 에러가 나오면 None 값을 반환하겠습니다.  

In [22]:
from hanspell import spell_checker
from pykospacing import Spacing
import datetime
import time
start_time = time.time()
err_cnt = 0
err_text = []
for i in tqdm(range(len(train))):
  result = None
  try:
    train_result = spell_checker.check(train['reviews'][i])
    result = train_result.checked
  except:
    try:
      spacing = Spacing()
      result = spacing(train['reviews'][i])
    except:
      err_cnt += 1
      err_text.append(train['reviews'][i])
      result = None
  train['reviews'][i] = result
end_time = time.time()
sec = (end_time-start_time)
result_time = datetime.timedelta(seconds=sec)

  0%|          | 0/149931 [00:00<?, ?it/s]C:\Users\User\anaconda3\envs\project4\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 149931/149931 [4:17:52<00:00,  9.69it/s]  


In [29]:
#시간 확인 및 에러 확인
print(result_time)
print(err_cnt)
for i in range(len(err_text)):
  print(err_text[i])

4:17:52.219570
0
0


훈련데이터 확인 및 저장

In [24]:
train['reviews']

0         사이즈를 3센티씩 늘린 건데도 작아요 그리고 색상은 정말 달라요 칙칙한 핑크네요ᅲᅲ...
1                          ᄇ 불만족.. 빗이 아픔.. 멍이 피부에 빗질 못해주겠네요
2         이 제품 쓰고 삼 일 만에 변기 물이 잘 안 내려갔어요. 혹시나 해서 다시 빼보니 ...
3                                                적당하고 만족합니다
4                         편하자고 이용하는 밀키 튼데 손은 근 많이 가서 저는 패스요
                                ...                        
149926    그냥그래요ㄷㄷㄷㄷㅂㅂㅂㅂ그냥그래요ㄷㄷㄷㄷㅂㅂㅂㅂㅁㅁㅁㅁㅋㅋㅋㅋㅌㅌㅌㅌㅍㅍㅍㅍㄹㄹㄹㄹ...
149927    비싸요 진짜 별거 아니게 허접하게 생겼는데 이게 이만 원이라니 진짜가 방 구겨지는 ...
149928                               100장 주문 안됩니다 500장 가능해요
149929            .하림치킨여기서구입하니엄청저렴하네요배송도쾅꽝얼어서도착하고아주만족합니다!ㅋㅋ
149930                                조금 약해 보이는데 저렴하게 잘 샀어요
Name: reviews, Length: 149931, dtype: object

In [31]:
train['reviews'].isnull().sum()

0

In [26]:
train.to_csv("preprocessing_train.csv", index=False)

테스트 데이터도 동일하게 수행합니다.

In [32]:
start_time = time.time()
err_cnt = 0
err_text = []
for i in tqdm(range(len(test))):
  result = None
  try:
    test_result = spell_checker.check(test['reviews'][i])
    result = test_result.checked
  except:
    try:
      spacing = Spacing()
      result = spacing(test['reviews'][i])
    except:
      err_cnt += 1
      err_text.append(test['reviews'][i])
      result = None
  test['reviews'][i] = result
end_time = time.time()
sec = (end_time-start_time)
result_time = datetime.timedelta(seconds=sec)

  0%|          | 0/49977 [00:00<?, ?it/s]C:\Users\User\anaconda3\envs\project4\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 49977/49977 [1:25:19<00:00,  9.76it/s]  


In [33]:
#시간 확인 및 에러 확인
print(result_time)
print(err_cnt)
for i in range(len(err_text)):
  print(err_text[i])

1:25:19.509589
0


In [34]:
test['reviews']

0                                      너무 낮고 솜도 적고 실망스럽습니다
1                                         피부에 뾰루지가 많이 올라와요
2         배송도 빠르네요 가격 대비 좋은 것 같아요~~~첨에는 힘들어 하나 조금 지나니 잘하네요
3                                 재구매입니다. 핏도 좋고 착용감도 좋습니다.
4          파손 제품 온 거 출장 같다 오늘 받았는데 현재 상황 장난하시는지 택배회사 항의하세요
                               ...                        
49972    연결해서 쓰려고 3단으로 2개 주문했는데.. 연결 부위 맞추기가 너무 힘드네요 거기...
49973    허리는 너무 크고 종아리는 꽉 끼어서 약간 불편해하네요 아들이 그래도 따뜻하다며 잘...
49974       샵에가서받았었는데이거구매하고집에서셀프로하는데넘나좋으네요^^*세트로사길잘한것같아요ㅎㅎ
49975    가격 도착하고 다 좋은데 조금 얇은 감이 있네요 넓이도 조금 커서 살짝 헐렁거리지만...
49976                                        아~정말 이건 아닌 듯ㅎ
Name: reviews, Length: 49977, dtype: object

In [35]:
test['reviews'].isnull().sum()

0

In [36]:
test.to_csv('preprocessing_test.csv', index=False)